In [ ]:
import numpy as np
import cv2
import copy
import os, matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.transforms import functional as F

import albumentations as A # Library for augmentations

import matplotlib.pyplot as plt

from tqdm import tqdm

In [ ]:
from MyUtils import transforms, utils, engine, train as transforms, utils, engine, train
from MyUtils.utils import collate_fn
from MyUtils.engine import train_one_epoch, evaluate
from MyUtils.plot_statistic import plot_stats
from MyUtils.visualize import visualize
import MyUtils.Dataset

In [ ]:
def train_transform():
    return A.Compose([
        A.OneOf([
        A.Sequential([
            A.RandomRotate90(p=0.5), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=(-0.2, 0.2), contrast_limit=(-0.2, 0.2), brightness_by_max=True, always_apply=False, p=0.5), # Random change of brightness & contrast
            #A.ToGray(p=1),
            #A.InvertImg(p=1),
            #A.ToGray(p=1)
            #A.Sharpen(alpha=(0.5, 0.5), p=1),
            A.RGBShift(r_shift_limit=20, g_shift_limit=20, b_shift_limit=20, always_apply=False, p=0.5)
            ]),
        A.Sequential([
            A.RandomScale(scale_limit=0.1, interpolation=1, always_apply=False, p=0.5)
        ]),
    ], p=1.0)
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

def eval_transform():
    return A.Compose([
        A.Sequential([
            #A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            #A.RandomBrightnessContrast(brightness_limit=(-0.5, 0.5), contrast_limit=(-0.2, 0.2), brightness_by_max=True, always_apply=False, p=1.0), # Random change of brightness & contrast
            #A.InvertImg(p=1),
            #A.ToGray(p=1)
            #A.Sharpen(alpha=(0.5, 0.5), p=1)
        ], p=1)
    ],
    keypoint_params=A.KeypointParams(format='xy'), # More about keypoint formats used in albumentations library read at https://albumentations.ai/docs/getting_started/keypoints_augmentation/
    bbox_params=A.BboxParams(format='pascal_voc', label_fields=['bboxes_labels']) # Bboxes should have labels, read more at https://albumentations.ai/docs/getting_started/bounding_boxes_augmentation/
    )

def test_transform():
    return A.Compose([
        A.Sequential([
            #A.RandomRotate90(p=1), # Random rotation of an image by 90 degrees zero or more times
            A.RandomBrightnessContrast(brightness_limit=(-0.5, 0.0), contrast_limit=0.0, brightness_by_max=True, always_apply=False, p=1.0), # Random change of brightness & contrast
            #A.InvertImg(p=1),
            #A.ToGray(p=1)
            #A.Sharpen(alpha=(0.1, 0.9), p=1)
        ], p=1)
    ])

In [ ]:
path = 'C:/Users/User/Petr/Net_3/Dataset/'

In [ ]:
dataset = MyUtils.Dataset.PillarsDataset(path, transform=None, demo=False)

dataset.explore

In [ ]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

iterator = iter(data_loader)
batch = next(iterator)

image = (batch[0][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes = batch[1][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()

idx = torch.IntTensor.item(batch[1][0]['image_id'])

print(f'ID: {dataset.inner_id}')

keypoints = []
for kps in batch[1][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints.append([kp for kp in kps])

print(bboxes)
visualize(image, bboxes, keypoints, save_path=os.path.join(path, 'check_dataset', '1.jpg'))

In [ ]:
dataset = MyUtils.Dataset.PillarsDataset(path, transform=train_transform(), demo=True)
dataset.explore

In [ ]:
data_loader = DataLoader(dataset, batch_size=1, shuffle=True, collate_fn=collate_fn)

iterator = iter(data_loader)
batch = next(iterator)

image = (batch[0][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes = batch[1][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()

keypoints = []
for kps in batch[1][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints.append([kp for kp in kps])

image_original = (batch[2][0].permute(1,2,0).numpy() * 255).astype(np.uint8)
bboxes_original = batch[3][0]['boxes'].detach().cpu().numpy().astype(np.int32).tolist()

keypoints_original = []
for kps in batch[3][0]['keypoints'].detach().cpu().numpy().astype(np.int32).tolist():
    keypoints_original.append([kp for kp in kps])

visualize(image, bboxes, keypoints, image_original, bboxes_original, keypoints_original, save_path=os.path.join(path, 'check_dataset', '1.jpg'))
print(f'ID: {dataset.inner_id}')
print(bboxes)
print(keypoints)

In [ ]:
def get_pillar_detection_model(num_classes, pretrain=True, weights_path=None):
    from torch import nn
    import torchvision
    from torchvision.models.detection import FasterRCNN
    from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
    from torchvision.models.detection.rpn import AnchorGenerator
    # load a model pre-trained pre-trained on COCO
    anchor_generator = AnchorGenerator(sizes=(32, 64, 128, 256, 512), aspect_ratios=(0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 2.0, 3.0, 4.0))
    
    if pretrain:
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn(weights='FasterRCNN_ResNet50_FPN_Weights.DEFAULT', min_size=1500, max_size=5000, class_weigth=[1.0, 3.0])
    else:
        model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes + 1) # lenght of classes WITHOUT background on input
    
    #model.rpn.anchor_generator = anchor_generator
    
#    model.roi_heads.box_predictor.cls_score = nn.Sequential(nn.Linear(1024, 512), nn.Linear(512,256), nn.Linear(256, 128), nn.Linear(128, 2))
    
#    model.roi_heads.box_predictor.bbox_pred = nn.Sequential(nn.Linear(1024, 512), nn.Linear(512,256), nn.Linear(256, 128), nn.Linear(128, 8))
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)
    return model

def get_model_V3(num_keypoints, weights_path=None):
    from torchvision.models.detection.keypoint_rcnn import KeypointRCNNPredictor
    from torchvision.models.detection.rpn import AnchorGenerator
    from torchvision.models.detection import keypointrcnn_resnet50_fpn
    # Load a pre-trained model
    model = keypointrcnn_resnet50_fpn(weights='DEFAULT')

# Replace the classifier head with the number of keypoints
    in_features = model.roi_heads.keypoint_predictor.kps_score_lowres.in_channels
    model.roi_heads.keypoint_predictor = KeypointRCNNPredictor(in_channels=in_features, num_keypoints=num_keypoints)

# Set the model's device and data type
    model.name = 'keypointrcnn_resnet50_fpn'
    if weights_path:
        state_dict = torch.load(weights_path)
        model.load_state_dict(state_dict)        
        
    return model

In [ ]:
#model = get_pillar_detection_model(num_classes=5)

#print(model)

In [ ]:
#model_path = 'C:/Users/User/Petr/Net_1/Source/save_model/model_weights_fourth_Alb_0.38.pth'
#classes = {'Building': 1}
#path = 'C:/Users/User/Petr/temp_images'

#model_d = load_model(model_path)

#backbone = torchvision.models.mobilenet_v2(weights='DEFAULT').features

In [ ]:
dataset_train = MyUtils.Dataset.PillarsDataset(path, transform=train_transform(), demo=False)
dataset_test = MyUtils.Dataset.PillarsDataset(path, transform=eval_transform(), demo=False)

dataset_train.explore
dataset_test.explore

points = dataset_train.get_max_corners()

In [ ]:
save_path = 'C:/Users/User/Petr/Net_3/save_model'
log_path = 'C:/Users/User/Petr/Net_3/Metric_log'

for num in range(1):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    indices = torch.randperm(len(dataset_train)).tolist()
    thirty_pc = int(len(dataset_train) * 0.20)
    dataset_train = torch.utils.data.Subset(dataset_train, indices[:-thirty_pc])
    dataset_test = torch.utils.data.Subset(dataset_test, indices[-thirty_pc:])

    data_loader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, collate_fn=collate_fn)
    data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

    model = get_model_V3(num_keypoints=points, weights_path=f'{save_path}/LS/weights_220.pth')#, weights_path=f'{save_path}/LS/weights_70.pth'


    model.to(device)

    params = [p for p in model.parameters() if p.requires_grad]
    #params1 = [p for p in model.roi_heads.keypoint_head.parameters() if p.requires_grad]
    
    optimizer = torch.optim.SGD(params, lr=1.8e-5, momentum=0.90)#, weight_decay=0.0001
    
    #optimizer = torch.optim.SGD([{'params': params1},
    #                             {'params': model.roi_heads.keypoint_predictor.parameters(), 'lr': .001},], lr=0.001, momentum=0.90)#, weight_decay=0.0001
    lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.8)
    num_epochs = 501

    kps_stats = []
    bbox_stats = []

    loss_bb = []
    loss_kp = []
    loss = []

    e_done = 220

    for epoch in range(e_done+1, num_epochs):
        logger = train_one_epoch(model, optimizer, data_loader_train, device, epoch, print_freq=20)
        lr_scheduler.step()
        evaluator = evaluate(model, data_loader_test, device)
        
        kps_stats.append(evaluator.coco_eval['keypoints'].stats[:6])
        bbox_stats.append(evaluator.coco_eval['bbox'].stats[:6])
    
        loss_bb.append(logger.meters['loss_box_reg'].global_avg)
        loss_kp.append(logger.meters['loss_keypoint'].global_avg)
        loss.append(logger.meters['loss'].global_avg)
        
        if epoch % 10 == 0 and epoch > 0:
            torch.save(model.state_dict(), f'{save_path}/LS/weights_{epoch}.pth')
            plot_stats(epoch - e_done, bbox_stats, kps_stats, loss_bb, loss_kp, loss, num=epoch, log_path=log_path)#, num=num
            
    plot_stats(num_epochs - e_done, bbox_stats, kps_stats, loss_bb, loss_kp, loss, num=epoch, show=True, log_path=log_path)

In [ ]:
test_path = 'C:/Users/User/Petr/Net_3/test_model/images/'
out_path = 'C:/Users/User/Petr/Net_3/test_model/pred/'

save_path = 'C:/Users/User/Petr/Net_3/save_model'
#model = get_model_V3(num_keypoints=points, weights_path=f'{save_path}/LS/weights_200.pth')

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

#model.to(device)


for image in tqdm(os.listdir(test_path)):
    img = cv2.imdecode(np.fromfile(os.path.join(test_path, image), dtype=np.uint8), cv2.IMREAD_UNCHANGED) #cv2.imread(os.path.join(test_path, image))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    img_orig = cv2.imdecode(np.fromfile(os.path.join(test_path, image), dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    img_orig = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    
    img = F.to_tensor(img)
    type(img)
    img = img.to(device)
    with torch.no_grad():
        #model_test.to(device)
        model.eval()
        out = model([img,])
        out = out[0]
    #img = (img[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
    scores = out['scores'].detach().cpu().numpy()
    kps_scores = out['keypoints_scores'].detach().cpu().numpy()

    high_scores_idxs = np.where(scores > 0.7)[0].tolist() # Indexes of boxes with scores > 0.7
    post_nms_idxs = torchvision.ops.nms(out['boxes'][high_scores_idxs], out['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

# Below, in output[0]['keypoints'][high_scores_idxs][post_nms_idxs] and output[0]['boxes'][high_scores_idxs][post_nms_idxs]
# Firstly, we choose only those objects, which have score above predefined threshold. This is done with choosing elements with [high_scores_idxs] indexes
# Secondly, we choose only those objects, which are left after NMS is applied. This is done with choosing elements with [post_nms_idxs] indexes
#    print(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0])
    if not len(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
        good_kps_idxs = [[]]
    else:
        good_kps_idxs = np.where(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0] > -2.0) # for i in range(len(post_nms_idxs))
#    print(good_kps_idxs[0])
#    print('KPs: ', out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0][good_kps_idxs[0]])
    #kps_scores_filtered = out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0][good_kps_idxs[0]]
#print(kps_scores_filtered)

#kps_filtered =  output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
#out['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
#print(kps_filtered)

    keypoints = []
    for kps in out['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
        keypoints.append([list(map(int, kp)) for kp in kps[good_kps_idxs[0]]])
    scores = kps_scores[0][good_kps_idxs[0]]
    #kps_scores = out['keypoints_scores'][high_scores_idxs][post_nms_idxs][good_kps_idxs[0]].detach().cpu().numpy()[0]
#kps_idxs = np.where(output[0]['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0] > 1.2)[0]
#print(kps_idxs)
#print(np.array(keypoints)[0][kps_idxs])
#keypoints = [np.array(keypoints)[0][kps_idxs].tolist()]
    #print(keypoints)
#keypoints = average_close_points(keypoints)
    #print(keypoints)
    #print(kps_scores[0][good_kps_idxs[0]])
    #print(high_scores_idxs, post_nms_idxs, good_kps_idxs[0])
    #break
    bboxes = []
    for bbox in out['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
        bboxes.append(list(map(int, bbox.tolist())))
    bld_scores = out['scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy().tolist()
    #print(bld_scores)
    #print(len(bboxes), len(bld_scores))
    #print(type(out[0]['boxes'].detach().cpu().numpy().tolist()))
    visualize(img_orig, bboxes, keypoints, bld_score=bld_scores, kps_scores=scores, save_path=out_path + image, obj_type='Pillar')

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model_V2(num_keypoints = 9, weights_path='./save_model/weights_new_dataset.pth')
model.to(device);
#print(model)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

path = 'C:/Users/User/Desktop/Teobox_Shipunovo_2022-10-16T21.03.14/export_shapes_2/shape_556/'
path_E = './check_model/images'

images = []



for num, label in enumerate(os.listdir(path_E)):
    #print(os.path.join(path_E, label))
    #if num >= 15:
    #    break
    if 'world' in label:
        continue
    img = cv2.imread(os.path.join(path_E, label))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = F.to_tensor(img)
    img = img.to(device)
    images.append((img, label))

images_t = []
for num, label in enumerate(os.listdir(path_E)):
    #if num >= 15:
    #    break
    if 'world' in label:
        continue
    img = cv2.imread(os.path.join(path_E, label))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #img = F.to_tensor(img)
    #img = img.to(device)
    images_t.append((img, label))
#print(type(images[0][0]))
images_transformed = []
for pair in images_t:
    #img = A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, brightness_by_max=True, always_apply=False, p=1)(image=pair[0])['image']
    #img = A.InvertImg(p=1)(image=img)['image']
    #img = A.Sharpen(alpha=(0.1, 0.9), p=1)(image=img)['image']
    img = test_transform()(image=pair[0])['image']
    img = F.to_tensor(img)
    img = img.to(device)
    images_transformed.append(img)
with torch.no_grad():
    model.to(device)
    model.eval()
    images_to_pred = images_transformed
    output = model(images_to_pred)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = get_model_V2(num_keypoints = 9, weights_path='./save_model/weights_rebuild_model_3.pth')
model.to(device);
#print(model)

In [ ]:
path = 'C:/Users/User/Desktop/Teobox_Shipunovo_2022-10-16T21.03.14/export_shapes/shape_401/'
path_E = 'C:/Users/User/Petr/Net_2/test_dataset/two_dataset/eval/images'

images = []



for num, label in enumerate(os.listdir(path)):
    if num >= 15:
        break
    if 'world' in label:
        continue
    img = cv2.imread(os.path.join(path, label))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = F.to_tensor(img)
    img = img.to(device)
    images.append((img, label))
#print(images)
with torch.no_grad():
    model.to(device)
    model.eval()
    images_to_pred = [image for (image, label) in images]
    output = model(images_to_pred)

In [ ]:
def get_object_detection_model(num_classes):
    from torch import nn
    import torchvision
    from torchvision.models.detection import FasterRCNN
    from torchvision.models.detection.rpn import AnchorGenerator
    from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
    
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes) # lenght of classes WITHOUT background on input
    
#    model.roi_heads.box_predictor.cls_score = nn.Sequential(nn.Linear(1024, 512), nn.Linear(512,256), nn.Linear(256, 128), nn.Linear(128, 2))
    
#    model.roi_heads.box_predictor.bbox_pred = nn.Sequential(nn.Linear(1024, 512), nn.Linear(512,256), nn.Linear(256, 128), nn.Linear(128, 8))

    return model


def load_model(filepath: str):   
    model = get_object_detection_model(num_classes=2)
    model.load_state_dict(torch.load(filepath))
    return model



In [ ]:
for image, out in zip(images, output):
    label = image[1][:-4] + 'big_ds'
    image = (image[0].permute(1,2,0).detach().cpu().numpy() * 255).astype(np.uint8)
    scores = out['scores'].detach().cpu().numpy()


    high_scores_idxs = np.where(scores > 0.75)[0].tolist() # Indexes of boxes with scores > 0.7
    post_nms_idxs = torchvision.ops.nms(out['boxes'][high_scores_idxs], out['scores'][high_scores_idxs], 0.3).cpu().numpy() # Indexes of boxes left after applying NMS (iou_threshold=0.3)

# Below, in output[0]['keypoints'][high_scores_idxs][post_nms_idxs] and output[0]['boxes'][high_scores_idxs][post_nms_idxs]
# Firstly, we choose only those objects, which have score above predefined threshold. This is done with choosing elements with [high_scores_idxs] indexes
# Secondly, we choose only those objects, which are left after NMS is applied. This is done with choosing elements with [post_nms_idxs] indexes
#    print(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0])
    if not len(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()):
        good_kps_idxs = [[]]
    else:
        good_kps_idxs = np.where(out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0] > 0.0) # for i in range(len(post_nms_idxs))
#    print(good_kps_idxs[0])
#    print('KPs: ', out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0][good_kps_idxs[0]])
    #kps_scores_filtered = out['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0][good_kps_idxs[0]]
#print(kps_scores_filtered)

#kps_filtered =  output[0]['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
#out['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()
#print(kps_filtered)

    keypoints = []
    for kps in out['keypoints'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
        keypoints.append([list(map(int, kp)) for kp in kps[good_kps_idxs[0]]])
#kps_idxs = np.where(output[0]['keypoints_scores'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy()[0] > 1.2)[0]
#print(kps_idxs)
#print(np.array(keypoints)[0][kps_idxs])
#keypoints = [np.array(keypoints)[0][kps_idxs].tolist()]
    print(keypoints)
#keypoints = average_close_points(keypoints)
#print(keypoints)
    
    bboxes = []
    for bbox in out['boxes'][high_scores_idxs][post_nms_idxs].detach().cpu().numpy():
        bboxes.append(list(map(int, bbox.tolist())))
    

    
    visualize(image, bboxes, keypoints)

In [ ]:
import shutil

for file in two_corner_files:
    #shutil.copy(f'./test_dataset/train/images/{file}.jpg', f'./test_dataset/two_dataset/train/images/{file}.jpg')
    #shutil.copy(f'./test_dataset/train/annotations/{file}.txt', f'./test_dataset/two_dataset/train/annotations/{file}.txt')
    
    shutil.copy(f'./test_dataset/eval/images/{file}.jpg', f'./test_dataset/two_dataset/eval/images/{file}.jpg')
    shutil.copy(f'./test_dataset/eval/annotations/{file}.txt', f'./test_dataset/two_dataset/eval/annotations/{file}.txt')
    
for file in three_corners_files:
    #shutil.copy(f'./test_dataset/train/images/{file}.jpg', f'./test_dataset/three_dataset/train/images/{file}.jpg')
    #shutil.copy(f'./test_dataset/train/annotations/{file}.txt', f'./test_dataset/three_dataset/train/annotations/{file}.txt')
    
    shutil.copy(f'./test_dataset/eval/images/{file}.jpg', f'./test_dataset/three_dataset/eval/images/{file}.jpg')
    shutil.copy(f'./test_dataset/eval/annotations/{file}.txt', f'./test_dataset/three_dataset/eval/annotations/{file}.txt')

In [ ]:
import shutil

for file in os.listdir('./annotations/'):
    if file[:-4] not in extra_annotations:
        shutil.copy(f'./annotations/{file}', f'./annotations_clear/{file}')

In [ ]:
#import numpy as np

def average_close_points(point_coords): 
    #point_coords = np.array([point[:-1] for point in point_coords[0]])
    point_coords = np.array(point_coords[0])
    maping = np.abs(point_coords[:, None, None] - point_coords[None, :, None]) <= 3
    print(maping)
    maping = maping.reshape(maping.shape[0], maping.shape[1], 3)
    print(maping)
    res = []
    for point in maping:
        res.append(np.average(point_coords, axis=0, weights=point))
    #point_coords = np.average(point_coords, axis=0, weights=maping)
    
    return [np.unique(res, axis=0).astype(int)]

In [ ]:
average_close_points(keypoints)

In [ ]:
from torchview import draw_graph

model_v = get_model(num_keypoints=4)

model_graph = draw_graph(model_v, input_size=(1, 3, 128, 128))
model_graph.visual_graph

In [ ]:
from torch import nn
from torch.nn import functional as Fnn

class MyModel(nn.Module):

    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16,kernel_size=(2,2),stride=1,padding='valid')
        self.conv1_1 = nn.Conv2d(in_channels=16, out_channels=32,kernel_size=(2,2),stride=1,padding='same')
        self.maxpool1 = nn.MaxPool2d(kernel_size=(2,2))
        self.conv2 = nn.Conv2d(in_channels=32,out_channels=64,kernel_size=(2,2),stride=1,padding='valid')
        self.conv2_2 = nn.Conv2d(in_channels=64,out_channels=32,kernel_size=(2,2),stride=1,padding='same')
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.dropout1 = nn.Dropout(0.5)
        self.linear1 = nn.Linear(in_features=32*149*149,out_features=500)
        self.linear2 = nn.Linear(in_features=500,out_features=250)
        self.dropout2 = nn.Dropout(0.5)
        self.linear3 = nn.Linear(in_features=250,out_features=18)


    def forward(self,x):
        #x = cv2.resize(x, (600, 600))
        x = Fnn.relu(self.conv1(x))
        x = Fnn.relu(self.conv1_1(x))
        x = self.maxpool1(x)
        x = Fnn.relu(self.conv2(x))
        x = Fnn.relu(self.conv2_2(x))
        x = self.maxpool2(x)
        #print(x.shape)
        x = x.view(-1,32*149*149) 
        x = self.dropout1(x)
        x = Fnn.relu(self.linear1(x))
        x = Fnn.relu(self.linear2(x))
        x = self.dropout2(x)
        x = self.linear3(x)
        return x


In [ ]:
model_self = MyModel()

criterion = nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adamax(model.parameters(), lr=0.001)
#device = torch.device('cuda')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model_self.to(device)
# Training loop
num_epochs = 5

KEYPOINTS_FOLDER_TRAIN = './NEW_DATASET'
KEYPOINTS_FOLDER_EVAL = './NEW_DATASET'

dataset_train = ClassDataset(KEYPOINTS_FOLDER_TRAIN, transform=train_transform(), demo=False)
dataset_test = ClassDataset(KEYPOINTS_FOLDER_EVAL, transform=None, demo=False)

data_loader_train = DataLoader(dataset_train, batch_size=1, shuffle=True, collate_fn=collate_fn)
data_loader_test = DataLoader(dataset_test, batch_size=1, shuffle=False, collate_fn=collate_fn)

for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(data_loader_train):
         # [[[x, y, v], [x, y, v], [0 , 0, 0], [0 , 0, 0], [0 , 0, 0], [0 , 0, 0], [0 , 0, 0], [0 , 0, 0], [0 , 0, 0], [0 , 0, 0]]]
        points = [point for point in labels[0]['keypoints'][0]]
        kps = []
        for point in points:
            x, y, v = point
            kps.append([(x-600)/600, (y-600)/600])
        #kps = [(x, y) for point in points for x, y, v in point]
        #kps.reshape(1, 8)
        #print(kps)
        #print(images[0].numpy())
        images = cv2.resize(images[0].numpy().transpose(1,2,0), (600, 600))
        #print(images)
        images = F.to_tensor(images).cuda()
        labels = torch.as_tensor(kps).cuda()
        # Clear the gradients
        optimizer.zero_grad()
        # Forward pass
        outputs = model_self(images).to(device)
        #print(outputs)
        labels = labels.view(-1, 18)
        #print(labels)
        loss = torch.sqrt(criterion(outputs, labels))
        # Backward pass and optimize
        loss.backward()
        optimizer.step()
        # Print Epoch, Step and Loss value.
        if (i+1) % 10 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, len(data_loader_train), loss.item()))

In [ ]:
import re
def has_cyrillic(text):
    return bool(re.search('[а-яА-Я]', text))

path = 'C:/Users/User/Petr/Net_2/test.jpg'

has_cyrillic(path)

In [ ]:
image = cv2.imdecode(np.fromfile(path, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(15,15))
plt.imshow(image)